## Import of libraries and data

In [ ]:
from traffic.core import Traffic
from traffic.data import navaids
from utils import helperfunctions as hf
import plotly.graph_objects as go
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import plotly.graph_objects as go

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
t = Traffic.from_file("/mnt/beegfs/store/Projects_CRM/STAR_paper/LSGG/processed/landing_all.parquet")
t = t.query("rwy == '04'")

***
## BELU3N
***

### Reduction to flights passing BELUS

In [ ]:
t_BELU3N = (
    t.iterate_lazy()
    .pipe(hf.aligned_navpoint, "BELUS")
    .eval(desc="aligned_BELUS", max_workers=20)
)
t_BELU3N.to_parquet(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/LSGG/processed/belus.parquet")

### Remove go-arounds

In [ ]:
t_BELU3N = (
    t_BELU3N.iterate_lazy()
    .pipe(hf.remove_ga, "LSGG")
    .eval(desc="removing GAs", max_workers=20)
)
t_BELU3N.to_parquet(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/LSGG/processed/belus_ga.parquet")

### Crop at threshold

In [ ]:
t_BELU3N = (
    t_BELU3N.iterate_lazy()
    .pipe(hf.crop_after_th, "LSGG", "04")
    .eval(desc="Cropping at TH", max_workers=20)
)
t_BELU3N.to_parquet(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/LSGG/processed/belus_ga_th.parquet")

### Crop before Waypoint

In [ ]:
t_BELU3N = (
    t_BELU3N.iterate_lazy()
    .pipe(hf.crop_before_wp, "BELUS")
    .eval(desc="cropped_before_BELUS", max_workers=20)
)
t_BELU3N.to_parquet(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/LSGG/processed/belus_ga_th_wp.parquet")

### Map-plot

In [ ]:
t_BELU3N = Traffic.from_file(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/LSGG/processed/belus_ga_th.parquet")

# Create a single figure for the map
fig = go.Figure()

for flight in t_BELU3N.sample(100):
    fig.add_trace(
        go.Scattermapbox(
            mode="lines",
            lat=flight.data["latitude"],
            lon=flight.data["longitude"],
            line=dict(width=1.5, color="#757ef3"),
            opacity=0.5,
            name="t_betos",
            showlegend=False,
            
        )
    )

# Add BELUS navaid
fig.add_trace(
    go.Scattermapbox(
        mode="markers+text",
        lat=[navaids["BELUS"].latitude],
        lon=[navaids["BELUS"].longitude],
        marker=dict(size=10, color="red"),
        text=["BELUS"],
        textposition="middle right",
        textfont=dict(color="red", size=25),
        name="BELUS",
        showlegend=False,
    )
)

# Add INDIS navaid
fig.add_trace(
    go.Scattermapbox(
        mode="markers+text",
        lat=[navaids["INDIS"].latitude],
        lon=[navaids["INDIS"].longitude],
        marker=dict(size=10, color="red"),
        text=["INDIS"],
        textposition="top left",
        textfont=dict(color="red", size=25),
        name="INDIS",
        showlegend=False,
    )
)

# Update layout for the map
fig.update_layout(
    width=1500,
    height=1000,
    margin=dict(l=20, r=20, t=20, b=20),
    mapbox=dict(
        style="carto-positron",
        zoom=8,
        center=dict(
            lat=46.23696078946708,
            lon=6.109110255543277,
        ),
    ),
)

# Show the figure
fig.show()


### Histogram

In [ ]:
t_BELU3N = Traffic.from_file(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/LSGG/processed/belus_ga_th_wp.parquet")

# Generate data for histogram --------------------------------------------------
# Empty lists
ids = []
distances = []
timestamp = []

for flight in tqdm(t_BELU3N):
    try:
        # determine distance form BELUS to RWY04 TH
        distance = (
            hf.crop_before_wp(flight, "BELUS")
            .cumulative_distance()
            .data.cumdist.iloc[-1]
        )
        # Append distance and is to lists
        distances.append(distance)
        ids.append(flight.flight_id)
        timestamp.append(flight.start)
    except:
        print(f"Nothing after BELUS for {flight.flight_id}")

# Turn lists into dataframe
df = pd.DataFrame(
    {"flight_id": ids, "distance": distances, "timestamp": timestamp}
)

# Calculate the 99th percentile
percentile_99 = np.percentile(df.distance, 99)

# Define the STAR distance
star_distance = 92.8

# Plot the histogram -----------------------------------------------------------
# Create the figure
fig = go.Figure()

# Add the histogram trace
fig.add_trace(
    go.Histogram(
        x=df.distance,
        xbins=dict(
            start=0,
            end=int(max(df.distance)),
            size=1,
        ),
        marker=dict(color="#1f77b4"),
        name="Observed distances",
        showlegend=True,
    ),
)

# Add a line indicating the 99th percentile
fig.add_shape(
    type="line",
    x0=percentile_99,
    x1=percentile_99,
    y0=0,
    y1=1,
    yref="paper",
    line=dict(
        color="#2ca02c",
        width=3,
    ),
)

fig.add_trace( # For proper legend
    go.Scatter(
        x=[percentile_99, percentile_99],
        y=[0, 1],
        mode="lines",
        line=dict(color="#2ca02c", width=3),
        name="99th Percentile of observed distances",
        showlegend=True,
    )
)

# Add the line for "Distance according to STAR"
fig.add_shape(
    type="line",
    x0=star_distance,
    x1=star_distance,
    y0=0,
    y1=1,
    yref="paper",
    line=dict(
        color="#d62728",
        width=3,
    ),
)

fig.add_trace( # For proper legend
    go.Scatter(
        x=[star_distance, star_distance],
        y=[0, 1],
        mode="lines",
        line=dict(color="#d62728", width=3),
        name="Distance according to STAR",
        showlegend=True,
        legendgroup="Histogram",
    )
)

# Add annotations for the STAR distance and the 99th percentile
fig.add_annotation(
    x=star_distance,
    y=-0.1,
    text=f"{star_distance}",
    showarrow=False,
    yref="paper",
    font=dict(size=25, color="#d62728"),
    xanchor="center",
)

fig.add_annotation(
    x=percentile_99,
    y=-0.1,
    text=f"{percentile_99:.2f}",
    showarrow=False,
    yref="paper",
    font=dict(size=25, color="#2ca02c"),
    xanchor="center",
)

# Update the x-axis
fig.update_xaxes(
    range=[25, 100],
    title_text="Distance flown from BELUS to THR RWY04 [NM]",
    titlefont=dict(size=30),
    tickfont=dict(size=25),
    title_standoff=40,
)

# Update the y-axis
fig.update_yaxes(
    title_text="Count",
    titlefont=dict(size=30),
    tickfont=dict(size=25),
    title_standoff=30,
)

# Update layout to place legend outside the plot area
fig.update_layout(
    width=2000,
    height=800,
    margin=dict(l=50, r=300, t=40, b=100),
    legend=dict(
        font=dict(size=30),
        yanchor="top",
        xanchor="left",
        x=1.02,
        y=1,
        tracegroupgap=10
    ),
)

# Show the figure
fig.show()


***
## KINE2N
***

### Reduction to flights passing KINES

In [ ]:
t_KINE2N = (
    t.iterate_lazy()
    .pipe(hf.aligned_navpoint, "KINES")
    .eval(desc="aligned_KINES", max_workers=20)
)
t_KINE2N.to_parquet("/mnt/beegfs/store/Projects_CRM/STAR_paper/LSGG/processed/kines.parquet")

### Remove go-arounds

In [ ]:
t_KINE2N = (
    t_KINE2N.iterate_lazy()
    .pipe(hf.remove_ga, "LSGG")
    .eval(desc="removing GAs", max_workers=20)
)
t_KINE2N.to_parquet(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/LSGG/processed/kines_ga.parquet")

### Crop at threshold

In [ ]:
t_KINE2N = (
    t_KINE2N.iterate_lazy()
    .pipe(hf.crop_after_th, "LSGG", "04")
    .eval(desc="Cropping at TH", max_workers=20)
)
t_KINE2N.to_parquet("/mnt/beegfs/store/Projects_CRM/STAR_paper/LSGG/processed/kines_ga_th.parquet")

### Crop before waypoint

In [ ]:
t_KINE2N = (
    t_KINE2N.iterate_lazy()
    .pipe(hf.crop_before_wp, "KINES")
    .eval(desc="cropped_before_KINES", max_workers=20)
)
t_KINE2N.to_parquet("/mnt/beegfs/store/Projects_CRM/STAR_paper/LSGG/processed/kines_ga_th_wp.parquet")

### Map-plot

In [ ]:
t_KINE2N = Traffic.from_file("/mnt/beegfs/store/Projects_CRM/STAR_paper/LSGG/processed/kines_ga_th.parquet")

# Create a single figure for the map
fig = go.Figure()

for flight in t_KINE2N.sample(100):
    fig.add_trace(
        go.Scattermapbox(
            mode="lines",
            lat=flight.data["latitude"],
            lon=flight.data["longitude"],
            line=dict(width=1.5, color="#757ef3"),
            opacity=0.5,
            name="t_betos",
            showlegend=False,
            
        )
    )

# Add KINES navaid
fig.add_trace(
    go.Scattermapbox(
        mode="markers+text",
        lat=[navaids["KINES"].latitude],
        lon=[navaids["KINES"].longitude],
        marker=dict(size=10, color="red"),
        text=["KINES"],
        textposition="middle right",
        textfont=dict(color="red", size=25),
        name="KINES",
        showlegend=False,
    )
)

# Add GOLEB navaid
fig.add_trace(
    go.Scattermapbox(
        mode="markers+text",
        lat=[navaids["GOLEB"].latitude],
        lon=[navaids["GOLEB"].longitude],
        marker=dict(size=10, color="red"),
        text=["GOLEB"],
        textposition="middle right",
        textfont=dict(color="red", size=25),
        name="GOLEB",
        showlegend=False,
    )
)

# Add BIVLO navaid
fig.add_trace(
    go.Scattermapbox(
        mode="markers+text",
        lat=[navaids["BIVLO"].latitude],
        lon=[navaids["BIVLO"].longitude],
        marker=dict(size=10, color="red"),
        text=["BIVLO"],
        textposition="top right",
        textfont=dict(color="red", size=25),
        name="BIVLO",
        showlegend=False,
    )
)

# Add BIVLO navaid
fig.add_trace(
    go.Scattermapbox(
        mode="markers+text",
        lat=[navaids.extent("Switzerland").get("GG502").latitude],
        lon=[navaids.extent("Switzerland").get("GG502").longitude],
        marker=dict(size=10, color="red"),
        text=["GG502"],
        textposition="bottom center",
        textfont=dict(color="red", size=25),
        name="GG502",
        showlegend=False,
    )
)

# Update layout for the map
fig.update_layout(
    width=1500,
    height=1000,
    margin=dict(l=20, r=20, t=20, b=20),
    mapbox=dict(
        style="carto-positron",
        zoom=8,
        center=dict(
            lat=46.23696078946708,
            lon=6.109110255543277,
        ),
    ),
)

# Show the figure
fig.show()

### Histogram

In [ ]:
t_KINE2N = Traffic.from_file(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/LSGG/processed/kines_ga_th_wp.parquet")

# Generate data for histogram --------------------------------------------------
# Empty lists
ids = []
distances = []
timestamp = []

for flight in tqdm(t_KINE2N):
    # determine distance form BELUS to RWY04 TH
    distance = (
        flight
        .cumulative_distance()
        .data.cumdist.iloc[-1]
    )
    # Append distance and is to lists
    distances.append(distance)
    ids.append(flight.flight_id)
    timestamp.append(flight.start)

# Turn lists into dataframe
df = pd.DataFrame(
    {"flight_id": ids, "distance": distances, "timestamp": timestamp}
)

# Calculate the 99th percentile
percentile_99 = np.percentile(df.distance, 99)

# Define the STAR distance
star_distance = 103.8

# Plot the histogram -----------------------------------------------------------
# Create the figure
fig = go.Figure()

# Add the histogram trace
fig.add_trace(
    go.Histogram(
        x=df.distance,
        xbins=dict(
            start=0,
            end=int(max(df.distance)),
            size=1,
        ),
        marker=dict(color="#1f77b4"),
        name="Observed distances",
        showlegend=True,
    ),
)

# Add a line indicating the 99th percentile
fig.add_shape(
    type="line",
    x0=percentile_99,
    x1=percentile_99,
    y0=0,
    y1=1,
    yref="paper",
    line=dict(
        color="#2ca02c",
        width=3,
    ),
)

fig.add_trace( # For proper legend
    go.Scatter(
        x=[percentile_99, percentile_99],
        y=[0, 1],
        mode="lines",
        line=dict(color="#2ca02c", width=3),
        name="99th Percentile of observed distances",
        showlegend=True,
    )
)

# Add the line for "Distance according to STAR"
fig.add_shape(
    type="line",
    x0=star_distance,
    x1=star_distance,
    y0=0,
    y1=1,
    yref="paper",
    line=dict(
        color="#d62728",
        width=3,
    ),
)

fig.add_trace( # For proper legend
    go.Scatter(
        x=[star_distance, star_distance],
        y=[0, 1],
        mode="lines",
        line=dict(color="#d62728", width=3),
        name="Distance according to STAR",
        showlegend=True,
        legendgroup="Histogram",
    )
)

# Add annotations for the STAR distance and the 99th percentile
fig.add_annotation(
    x=star_distance,
    y=-0.1,
    text=f"{star_distance}",
    showarrow=False,
    yref="paper",
    font=dict(size=25, color="#d62728"),
    xanchor="center",
)

fig.add_annotation(
    x=percentile_99,
    y=-0.1,
    text=f"{percentile_99:.2f}",
    showarrow=False,
    yref="paper",
    font=dict(size=25, color="#2ca02c"),
    xanchor="center",
)

# Update the x-axis
fig.update_xaxes(
    range=[40, 150],
    title_text="Distance flown from KINES to THR RWY04 [NM]",
    titlefont=dict(size=30),
    tickfont=dict(size=25),
    title_standoff=40,
)

# Update the y-axis
fig.update_yaxes(
    title_text="Count",
    titlefont=dict(size=30),
    tickfont=dict(size=25),
    title_standoff=30,
)

# Update layout to place legend outside the plot area
fig.update_layout(
    width=2000,
    height=800,
    margin=dict(l=50, r=300, t=40, b=100),
    legend=dict(
        font=dict(size=30),
        yanchor="top",
        xanchor="left",
        x=1.02,
        y=1,
        tracegroupgap=10
    ),
)

# Show the figure
fig.show()
